---
LSTM to generate City names
---

Here, I am duplicating the work by 'Sirajology' as you can find it under https://www.youtube.com/watch?v=KvoZU-ItDiE
This is the first time I do something with tflearn. I must say, it makes rapid prototyping possible, but really abstracts all concepts lying below... Not sure if this such a good thing :)

First, import all dependencies. Just in case you have troubles installing tflearn (as I did), we install it if not installed.

In [1]:
#!/Library/Frameworks/Python.framework/Versions/2.7/bin/python
from __future__ import absolute_import, division, print_function

import os
from six import moves
import ssl


try:
    import tflearn
except ImportError:
    import pip
    pip.main(['install', 'tflearn'])

import tflearn
from tflearn.data_utils import *


Retrieve the data. Well, not much to add to this.


In [2]:
path = "US_cities.txt"
if not os.path.isfile(path):
    context = ssl._create_unverified_context()
    moves.urllib.request.urlretrieve("https://raw.githubusercontent.com/tflearn/tflearn.github.io/master/resources/US_Cities.txt", 
                                    path, context=context)

Defining parameters. This is how long the resulting city-names are supposed to be.

In [3]:
maxlen = 20

Preprocess data. We must turn characters into some kind of vectors because you can calculate numbers for gradients only... Well, computers use numbers so we need to adapt to their language.

In [4]:
#vectorize  text file
X, Y, char_idx = textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3)

Vectorizing text...
Text total length: 207041
Distinct chars: 61
Total sequences: 69007


Defining the model. This should be self-explanatory

In [5]:
g = tflearn.input_data(shape=[None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)


Generate cities. We use the SequenceGenerator module given by tflearn to do this. We save the current state of the model under 'model_us_cities' in case something goes wrong during training. We clip the gradients for RNN's because they can explode or collapse otherwise.

In [6]:
m = tflearn.SequenceGenerator(g, 
                              dictionary=char_idx, 
                              seq_maxlen=maxlen, 
                              clip_gradients=5.0,
                              checkpoint_path="model_us_cities")

Now we're training the model. We train, and at each step, we generate what the model would generate to show the process the model evolves through.

In [8]:
for i in xrange(40):
    seed = random_sequence_from_textfile(path, maxlen)
    m.fit(X, Y, validation_set=0.1, batch_size=128, n_epoch=1, run_id='us cities')
    print("-- Test with temperature of 1.2 --")
    print(m.generate(30, temperature=1.2, seq_seed=seed))
    print("-- Test with temperature of 1.0 --")
    print(m.generate(30, temperature=1.0, seq_seed=seed))
    print("-- Test with temperature of 0.5 --")
    print(m.generate(30, temperature=0.5, seq_seed=seed)) #figure out what temperature is... (from a mathematical perspective...)

Training Step: 1515  | total loss: 2.15830
| Adam | epoch: 000 | loss: 2.15830 -- iter: 00768/62106


KeyboardInterrupt: 